In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import numpy as np

import tensorflow as tf

from data.make_adience_dataset import build_dataset
from model.build_model import LeviHassnerBackbone, MultiTaskHead
from utils.train_multitask import train_model_multitask
from utils.train_singletask import train_model_singletask

In [2]:
# train_ds, val_ds = build_dataset( "/export/home/aneezahm001/nndl_2/backups/adience_gender/test_fold_is_0") #load gender dataset
# train_ds, val_ds = build_dataset( "/export/home/aneezahm001/nndl_2/adience/adience_age/test_fold_is_0") #load age dataset

train_ds, val_ds = build_dataset( "/export/home/aneezahm001/nndl_2/adience/adience_age_gender/test_fold_is_0", batch_size=8) #load age and gender combined dataset
# train_ds, val_ds = build_dataset("/export/home/aneezahm001/nndl_2/celeba-4/output", batch_size=128)

TFRecord files for training: 
 ['/export/home/aneezahm001/nndl_2/adience/adience_age_gender/test_fold_is_0/train-00006-of-00010', '/export/home/aneezahm001/nndl_2/adience/adience_age_gender/test_fold_is_0/train-00000-of-00010', '/export/home/aneezahm001/nndl_2/adience/adience_age_gender/test_fold_is_0/train-00003-of-00010', '/export/home/aneezahm001/nndl_2/adience/adience_age_gender/test_fold_is_0/train-00004-of-00010', '/export/home/aneezahm001/nndl_2/adience/adience_age_gender/test_fold_is_0/train-00002-of-00010', '/export/home/aneezahm001/nndl_2/adience/adience_age_gender/test_fold_is_0/train-00009-of-00010', '/export/home/aneezahm001/nndl_2/adience/adience_age_gender/test_fold_is_0/train-00008-of-00010', '/export/home/aneezahm001/nndl_2/adience/adience_age_gender/test_fold_is_0/train-00005-of-00010', '/export/home/aneezahm001/nndl_2/adience/adience_age_gender/test_fold_is_0/train-00007-of-00010', '/export/home/aneezahm001/nndl_2/adience/adience_age_gender/test_fold_is_0/train-00001

In [3]:
from matplotlib import pyplot as plt
for x, y, z in train_ds.take(5):
    print(x.shape)
    # plt.imshow(x[0])
    # plt.show()

(8, 227, 227, 3)
(8, 227, 227, 3)
(8, 227, 227, 3)
(8, 227, 227, 3)
(8, 227, 227, 3)


In [ ]:
train = "./Dataset/Train"
test = "./Dataset/Test"
valid = "./Dataset/Validation"

In [ ]:
# #build and train inception model
inception = tf.keras.applications.inception_v3.InceptionV3(
    include_top=False,
    weights="imagenet",
    input_shape=(227, 227, 3),
    # classes=2,
    # classifier_activation="softmax",
)

In [ ]:
from tensorflow.keras import Sequential
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, BatchNormalization, Dropout , GlobalAveragePooling2D
inception = LeviHassnerBackbone(include_head=True, num_classes=2, weight_decay=5e-4, initializer="levi_hassner") #8 age classes with single head

model = Sequential([inception,
                   GlobalAveragePooling2D(),
                   BatchNormalization(),
                   Dense(256,activation='relu'),
                   BatchNormalization(),
                  #  Dropout(0.5),
                #    Dense(128, activation="relu"),
                #    BatchNormalization(),
                #    Dropout(0.5),
                   Dense(2, activation="softmax")]
                   )

# model = Sequential([inception,
#                    GlobalAveragePooling2D(),
#                    BatchNormalization(),
#                    Dense(256,activation='relu'),
#                    BatchNormalization(),
#                    Dropout(0.5),
#                    Dense(128, activation="relu"),
#                    BatchNormalization(),
#                    Dropout(0.5),
#                    Dense(2,activation='softmax')])
# model.summary()

In [ ]:
inception.build(input_shape=(None, 227, 227, 3))
inception.summary()

In [ ]:
print(str(inception.summary()))

In [ ]:
inception.summary(print_fn=lambda x: print(x + '\n'))

In [ ]:
inception.load_weights("./checkpoints_gender/pretrained_gender")


In [ ]:
for layer in inception.layers:
    if "Conv" in layer.name:
        layer.trainable = False

In [ ]:
inception.summary()

In [ ]:
inception.build(input_shape=(None, 227, 227, 3))

In [ ]:
str(inception.summary())

In [ ]:
model2.layers[0].save_weights("test_checkpoint")

In [ ]:
inception2.load_weights("test_checkpoint")

In [ ]:
inception.compile(optimizer="sgd",loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['sparse_categorical_accuracy'])
# inception.build(input_shape=(None, 227, 227, 3))

In [ ]:
history = inception.fit(train_ds,epochs=5,validation_data=val_ds)

In [ ]:
inception.evaluate(valid_ds)

In [ ]:
inception.summary()

In [ ]:
train_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255.0,zoom_range=0.2,shear_range=0.2)
test_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255.0)
valid_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255.0)

In [ ]:
train_ds = train_gen.flow_from_directory(train,target_size=(227, 227),batch_size=32)
test_ds = test_gen.flow_from_directory(test,target_size=(227, 227),batch_size=32)
valid_ds = valid_gen.flow_from_directory(valid,target_size=(227, 227),batch_size=32)

In [ ]:
train_ds

In [ ]:
train_ds

In [ ]:
from tensorflow import keras
gender = keras.Sequential([
                          keras.layers.Conv2D(96, 7, strides=4, padding='valid', activation='relu', input_shape=(227, 227,3)),
                          keras.layers.MaxPooling2D(3, 2, padding='valid'),
                          keras.layers.BatchNormalization(axis=1, epsilon=0.001, momentum=0.9997),
                          keras.layers.Conv2D(256, 5, strides=1, padding='same', activation='relu'),
                          keras.layers.MaxPooling2D(3, 2, padding='valid'),
                          keras.layers.BatchNormalization(axis=1, epsilon=0.001, momentum=0.9997),
                          keras.layers.Conv2D(384, 3, strides=1, padding='valid', activation='relu'),
                          keras.layers.MaxPooling2D(3, 2, padding='valid'),
                          keras.layers.Flatten(),
                          keras.layers.Dense(512, activation='relu'),
                          keras.layers.Dropout(0.5),
                          keras.layers.Dense(512, activation='relu'),
                          keras.layers.Dropout(0.5),
                          keras.layers.Dense(2, activation='softmax')
])
gender.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=[tf.keras.metrics.CategoricalAccuracy()])

In [ ]:
gender.fit(train_ds, validation_data=valid_ds, epochs=80,)

In [ ]:
for x, y, z in train_ds.take(100):
    for y_ind in y:
        if y_ind == 6:
            print(y_ind)

In [ ]:
inception = train_model_singletask(
    inception, train_ds, val_ds, task="gender",
    # loss_fn=tf.keras.losses.BinaryCrossentropy(), 
    # acc_metric=tf.keras.metrics.BinaryAccuracy(),
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(),
    acc_metric=tf.keras.metrics.SparseCategoricalAccuracy(),
    # alpha=3e-1,
    # optimizer="sgd",
)

In [ ]:
inception.save_weights("./checkpoints_gender/pretrained_gender")

In [ ]:
inception.summary()

In [ ]:
#Build single task model for age detection

# train_model_singletask(
#     model, train_ds, val_ds, task="gender", 
#     loss_fn=tf.keras.losses.BinaryCrossentropy(), 
#     acc_metric=tf.keras.metrics.BinaryAccuracy(),
#     # loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(),
#     # acc_metric=tf.keras.metrics.SparseCategoricalAccuracy(),
#     alpha=3e-1,
#     optimizer="sgd",
# )

In [ ]:
#Build and train multi-task model
input = tf.keras.layers.Input(shape=(227, 227, 3))
backbone = LeviHassnerBackbone(initializer = "None") #Load model
head = MultiTaskHead()
latent = backbone(input)
output = head(latent)
model = tf.keras.Model(inputs=input, outputs=output)

model = train_model_multitask(
    model, 
    train_ds, 
    val_ds,
    epochs=10,
    
)

In [ ]:
model.layers[1].save_weights("./checkpoints/pretrained_age_gender")

# inception.save_weights("./checkpoints/pretrained_age_gender")

In [ ]:
#Build single task model for gender detection
model = LeviHassnerBackbone(include_head=True, num_classes=1) #binary classification with single head
train_model_singletask(
    model, train_ds, val_ds, task="gender", 
    loss_fn=tf.keras.losses.BinaryCrossentropy(), 
    acc_metric=tf.keras.metrics.BinaryAccuracy(),
)

In [ ]:
#Build single task model for age detection
model = LeviHassnerBackbone(include_head=True, num_classes=8) #8 age classes with single head
train_model_singletask(
    model, train_ds, val_ds, task="age", 
    loss_fn=tf.keras.losses.SparseCategoricalCrossentropy(), 
    acc_metric=tf.keras.metrics.SparseCategoricalAccuracy(),
)